# 3.

* Heap 클래스 (교재 코드): 최대 힙(Max Heap)을 구현한 클래스이다. 삽입 시 percolateUp, 삭제 시 percolateDown을 통해 힙 속성을 유지한다. 내부 리스트 __A는 힙의 요소들을 저장한다.

* 생일 데이터 전처리: pandas를 사용하여 CSV 파일에서 생일 데이터를 읽고, 생년월일이 비어 있는 행은 제거한 후 정수형으로 변환한다.

* 힙에 삽입: (생년월일, 이름) 튜플을 힙에 삽입한다. 최대 힙이므로 생일이 가장 늦은 친구가 루트에 위치하게 된다.

* 최신 생일 10명 출력: deleteMax()를 통해 생일이 가장 늦은 친구 10명을 차례로 출력한다.

In [1]:
class Heap:
	def __init__(self, *args):
		if len(args) != 0:
			self.__A = args[0] # 파라미터로 온 리스트
		else:
			self.__A = []
 
	# [알고리즘 8-2] 구현: 힙에 원소 삽입하기(재귀 알고리즘 버전)
	def insert(self, x):
		self.__A.append(x)
		self.__percolateUp(len(self.__A)-1)

	# 스며오르기
	def __percolateUp(self, i:int):
		parent = (i - 1) // 2
		if i > 0 and self.__A[i] > self.__A[parent]:
			self.__A[i], self.__A[parent] = self.__A[parent], self.__A[i]
			self.__percolateUp(parent)

	# [알고리즘 8-2] 구현: 힙에서 원소 삭제하기
	def deleteMax(self):
		# heap is in self.__A[0...len(self.__A)-1]
		if (not self.isEmpty()):
			max = self.__A[0]
			self.__A[0] = self.__A.pop() # *.pop(): 리스트의 끝원소 삭제 후 원소 리턴
			self.__percolateDown(0)
			return max
		else:
			return None

	# 스며내리기
	def __percolateDown(self, i:int):
		# Percolate down w/ self.__A[i] as the root
		child = 2 * i + 1  # left child
		right = 2 * i + 2  # right child
		if (child <= len(self.__A)-1):
			if (right <= len(self.__A)-1 and self.__A[child] < self.__A[right]):
				child = right  # index of larger child
			if self.__A[i] < self.__A[child]:
				self.__A[i], self.__A[child] = self.__A[child], self.__A[i]
				self.__percolateDown(child)

	def max(self):
		return self.__A[0]

	# 힙 만들기
	def buildHeap(self):
		for i in range((len(self.__A) - 2) // 2, -1, -1):
			self.__percolateDown(i)

	# 힙이 비었는지 확인하기
	def isEmpty(self) -> bool:
		return len(self.__A) == 0

	def clear(self):
		self.__A = []

	def size(self) -> int:
		return len(self.__A)


import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("C:/Users/sinji/data structures/birthday.csv", encoding="cp949")

# 생년월일이 비어있는 행 제거
df = df.dropna(subset=["생년월일8자리(예.20040101)"])

# 생년월일을 정수형으로 변환
df["생년월일8자리(예.20040101)"] = df["생년월일8자리(예.20040101)"].astype(int)

# 힙 객체 생성
h = Heap()

# (생년월일, 이름) 튜플로 힙에 삽입
for _, row in df.iterrows():
    name = row["이름"]
    birth = row["생년월일8자리(예.20040101)"]
    h.insert((birth, name))

# 생일이 가장 늦은 10명 출력
print("생일이 늦은 친구 10명:")
for _ in range(min(10, h.size())):
    birth, name = h.deleteMax()
    print(f"{name}: {birth}")



생일이 늦은 친구 10명:
홍서연: 20241282
신수민: 20051230
이서영: 20051225
강민주: 20051214
김민경: 20051202
이서영: 20051112
배시은: 20051102
김여원: 20051031
이서진: 20051028
서홍빈: 20051024


# 4.

* BidirectNode 클래스(교재 코드): 이 클래스는 이중 연결 리스트의 노드를 나타내며, 각 노드는 데이터를 저장하고 이전/다음 노드를 참조한다.

* CircularDoublyLinkedList 클래스(교재 코드): 원형 이중 연결 리스트로, __head는 더미 헤드를 가리킨다. 이 리스트는 순환 구조로 끝에 도달하면 다시 처음으로 돌아간다.

* CircularDoublyLinkedListIterator 클래스(교재 코드): CircularDoublyLinkedList를 순차적으로 탐색할 수 있는 반복자(Iterator)이다.

* 생일 데이터 로딩: pandas를 이용해 CSV 파일에서 생일 데이터를 읽고, CircularDoublyLinkedList에 저장한다.

* 조원 목록 필터링: my_team에 해당하는 조원들의 생일만 출력하며, 생일 정보가 없는 조원들은 제외한다.

In [2]:
class BidirectNode:
    def __init__(self, x, prevNode:'BidirectNode', nextNode:'BidirectNode'):
        self.item = x
        self.prev = prevNode
        self.next = nextNode
        
# 코드 5-24

class CircularDoublyLinkedList:
	def __init__(self):
		self.__head = BidirectNode("dummy", None, None)
		self.__head.prev = self.__head
		self.__head.next = self.__head
		self.__numItems = 0
 
	def insert(self, i:int, newItem) -> None:
		if (i >= 0 and i <= self.__numItems):
			prev = self.getNode(i - 1)
			newNode = BidirectNode(newItem, prev, prev.next)
			newNode.next.prev = newNode
			prev.next = newNode
			self.__numItems += 1
		else:
			print("index", i, ": out of bound in insert()") # 필요 시 에러 처리

	def append(self, newItem) -> None:
		prev = self.__head.prev
		newNode = BidirectNode(newItem, prev, self.__head)
		prev.next = newNode
		self.__head.prev = newNode
		self.__numItems += 1

	def pop(self, *args):
		# 가변 파라미터. 인자가 없거나 -1이면 마지막 원소로 처리하기 위함. 파이썬 리스트 규칙 만족
		if self.isEmpty():
			return None
		# 인덱스 i 결정
		if len(args) != 0: # pop(k)과 같이 인자가 있으면 i = k 할당
			i = args[0]
		if len(args) == 0 or i == -1:# pop()에 인자가 없거나 pop(-1)이면 i에 맨 끝 인덱스 할당
			i = self.__numItems - 1
		# i번 원소 삭제
		if (i >= 0 and i <= self.__numItems - 1):
			curr = self.getNode(i)
			retItem = curr.item
			curr.prev.next = curr.next
			curr.next.prev = curr.prev
			self.__numItems -= 1
			return retItem
		else:
			return None
 
	def remove(self, x):
		curr = self.__findNode(x)
		if curr != None:
			curr.prev.next = curr.next
			curr.next.prev = curr.prev
			self.__numItems -= 1
			return x
		else:
			return None

	def get(self, *args):
		# 가변 파라미터. 인자가 없거나 -1이면 마지막 원소로 처리하기 위함. 파이썬 리스트 규칙 만족
		if self.isEmpty(): return None
		# 인덱스 i 결정
		if len(args) != 0:   # pop(k)과 같이 인자가 있으면 i = k 할당
			i = args[0]
		if len(args) == 0 or i == -1: # pop()에 인자가 없거나 pop(-1)이면 i에 맨 끝 인덱스 할당
			i = self.__numItems - 1
		# i번 원소 리턴
		if (i >= 0 and i <= self.__numItems - 1):
			return self.getNode(i).item
		else:
			return None
 
	def index(self, x) -> int:
		cnt = 0
		for element in self:
			if element == x:
				return cnt
			cnt += 1
		return -12345

	def isEmpty(self) -> bool:
		return self.__numItems == 0
 
	def size(self) -> int:
		return self.__numItems

	def clear(self):
		self.__head = BidirectNode("dummy", None, None)
		self.__head.prev = self.__head
		self.__head.next = self.__head
		self.__numItems = 0
 
	def count(self, x) -> int:
		cnt = 0
		for element in self:
			if element == x:
					cnt += 1
		return cnt

	def extend(self, a): # a는 순회 가능한 모든 객체
		for element in a:
			self.append(element)

	def copy(self) -> 'CircularDoublyLinkedList':
		a = CircularDoublyLinkedList()
		for element in self:
			a.append(element)
		return a
 
	def reverse(self) -> None:
		prev = self.__head; curr = prev.next; next = curr.next
		self.__head.next = prev.prev; self.__head.prev = curr
		for i in range(self.__numItems):
			curr.next = prev; curr.prev = next
			prev = curr; curr = next; next = next.next
 
	def sort(self) -> None:
		a = []
		for element in self:
			a.append(element)
		a.sort()
		self.clear()
		for element in a:
			self.append(element)

	def __findNode(self, x) -> BidirectNode:
		curr = self.__head.next  # 0번 노드
		while curr != self.__head:
 
			if curr.item == x:
				return curr
			else:
				curr = curr.next
		return None

	def getNode(self, i:int) -> BidirectNode:
		curr = self.__head  # 더미 헤드, index: -1
		for index in range(i + 1):
			curr = curr.next
		return curr

	def printList(self) -> None:
		for element in self:
			print(element, end = ' ')
		print()
 
	def __iter__(self):  # generating iterator and return
		return CircularDoublyLinkedListIterator(self)
 
class CircularDoublyLinkedListIterator:
	def __init__(self, alist):
		self.__head = alist.getNode(-1)  		# 더미 헤드
		self.iterPosition = self.__head.next  	# 0번 노드
	def __next__(self):
		if self.iterPosition == self.__head: 	# 순환 끝
			raise StopIteration
		else: # 현재 원소를 리턴하면서 다음 원소로 이동
			item = self.iterPosition.item
			self.iterPosition = self.iterPosition.next
			return item

# 코드 5-25


import pandas as pd

# 1. 생일 데이터 로딩 (cp949)
df = pd.read_csv("C:/Users/sinji/data structures/birthday.csv", encoding="cp949")

# 2. CircularDoublyLinkedList에 데이터 삽입
birth_list = CircularDoublyLinkedList()

for idx, row in df.iterrows():
    if pd.isna(row['생년월일8자리(예.20040101)']):
        continue  # 생일이 비어있는 경우는 스킵
    birth_list.append({
        '학번': row['학번'],
        '이름': row['이름'],
        '생년월일': str(int(row['생년월일8자리(예.20040101)']))
    })

# 3. 같은 조 조원 목록
my_team = ['송윤경', '김혜정', '최지안', '정세원', '김정민', '이정원', '신다연', '이우정', '남궁수아', '신지예']

# 4. 출력
# 송윤경, 최지안, 이정원은 생일이 비어있어 제외
print("내 조 친구들의 생일: \n")
for person in birth_list:
    if person['이름'] in my_team:
        print(f"{person['이름']}: {person['생년월일']}")


내 조 친구들의 생일: 

김정민: 20050422
김혜정: 20050501
남궁수아: 20050325
신다연: 20041206
신지예: 20040707
이우정: 20020324
정세원: 20041121


# 5. 교재 연습문제

## Problem 01

가질 수 있다.

최대 힙은 부모노드가 자식노드보다 크다는 조건을 만족하면 된다. 형제 노드나 서로 다른 서브트리 간의 관계는 보장되지 않기 때문에, 더 깊은 곳에 있는 원소가 더 얕은 곳에 있는 원소보다 클 수 있다.

## Problem 02

항상 그렇지 않다.

A[n-1]은 마지막 인덱스에 있는 노드일 뿐이다. 최대 힙의 규칙은 부모 >= 자식이라는 조건만 만족하면 되므로, 맨 마지막 원소가 항상 최소라는 보장은 없다.

## Problem 03

n/2를 올림한 개수

buildHeap()은 리스트의 중간부터 역순으로 내려가며 스며내리기를 수행한다. 리프 노드는 자식이 없으므로 스며내리기를 수행할 필요가 없다. 완전 이진 트리에서 리프 노드는 n/2를 내림한 인덱스부터 n-1번 인덱스에 존재하므로, 스며내리기를 하지 않는 노드 수는 n/2를 올림한 개수와 같다.

## Problem 04

최악의 경우에 해당하는 시간: Θ(log n)

최선의 경우에 해당하는 시간: Θ(1)

한 번의 스며내리기에서 루트부터 리프까지 내려갈 수 있으므로 최대 깊이는 log₂n이다.
최악의 경우 모든 단계에서 교환이 일어나는 경우로 Θ-표기로 나타내면 Θ(log n)이다.
최선의 경우 한 번도 교환 없이 종료되는 경우로 Θ-표기로 나타내면 Θ(1)이다.

## Problem 05

아니요.

힙은 완전 이진 트리 구조를 유지해야 하기 때문에, 힙의 맨 마지막 원소를 삭제하는 것만으로는 트리의 구조가 깨질 수 있어서 간단하지 않다.

## Problem 06

아니요. 스며내리기의 점근적 시간은 O(n)이고 스며오르기의 점근적 시간은 O(n log n)로 스며내리기 방식이 더 효율적이다. 

스며내리기 방식은 리프 노드부터 시작해서 높이가 작은 서브트리부터 힙 구조를 만들어가며 진행되므로 전체 시간 복잡도는 O(n)이다. 스며오르기 방식은 노드를 하나씩 추가하면서 위로 올리기 때문에 일반적으로 insert() 반복이라서 점근적 시간은 O(n log n)이다.

## Problem 07

값이 증가한 노드는 부모와 비교하며 위로 올라가는 방식으로 최대 O(log n) 시간에 힙을 수선할 수 있다.

어떤 원소의 값이 증가하면, 그 값이 부모 노드보다 커질 수 있다. 이 경우 힙의 성질인 부모 >= 자식을 만족하지 않게 되므로 해당 원소를 부모와 비교하면서 위로 이동시키는 과정을 수행해야 한다.

과정:
1. 해당 원소의 인덱스를 i라고 한다.
2. 부모 인덱스는 parent(i) = (i-1)//2로 계산된다.
3. A[i] > A[parent(i)]이면 두 값을 교환한다.
4. 이 과정을 현재 노드가 루트에 도달하거나, 부모보다 작거나 같을 때까지 반복한다.
5. 힙의 높이는 O(log n)이므로, 이 과정은 최악의 경우 O(log n) 시간 안에 종료된다.

# 6. LeetCode 703

* __init__(k, nums): 최소 힙을 만들고 nums를 넣어 정리함.

* add(val): 새 값을 넣고, 힙의 크기가 k 초과 시 작은 값 제거 → k번째로 큰 수 반환.

* self.heap[0]: 항상 현재까지의 k번째로 큰 수가 위치함.

In [3]:
import heapq  # 파이썬 기본 힙 모듈 (min heap)

class KthLargest:
    def __init__(self, k, nums):
        self.k = k
        self.heap = []  # 힙에 k개의 큰 수만 유지할 거야

        for num in nums:
            self.add(num)  # 초기 숫자도 하나씩 넣어서 관리

    def add(self, val):
        heapq.heappush(self.heap, val)  # 새 수 넣기
        if len(self.heap) > self.k:
            heapq.heappop(self.heap)    # 너무 많으면 제일 작은 거 빼기
        return self.heap[0]     

In [4]:
kthLargest = KthLargest(3, [4, 5, 8, 2])
print(kthLargest.add(3))  # return 4
print(kthLargest.add(5))  # return 5
print(kthLargest.add(10)) # return 5
print(kthLargest.add(9))  # return 8
print(kthLargest.add(4))  # return 8

4
5
5
8
8
